In [1]:
import os

from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.STEPCAFControl import STEPCAFControl_Reader
from OCC.Core.TCollection import TCollection_ExtendedString
from OCC.Core.TDocStd import TDocStd_Document
from OCC.Core.VrmlAPI import VrmlAPI_Writer
from OCC.Core.XCAFDoc import XCAFDoc_ShapeTool, XCAFDoc_DocumentTool
from OCC.Core.BRep import BRep_Tool
from OCC.Core.TDF import TDF_LabelSequence, TDF_Label
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.TopAbs import topabs
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
import xml.etree.ElementTree as ET
from IPython.display import HTML

In [2]:
stp_filename = 'VentilatorAP203' #'as1_pe_203'
stp_path = os.path.join('..','assets', 'step', stp_filename + '.step')

# load the STEP file
doc = TDocStd_Document(TCollection_ExtendedString("pythonocc-doc"))

step_reader = STEPCAFControl_Reader()
step_reader.SetColorMode(True)
step_reader.SetLayerMode(True)
step_reader.SetNameMode(True)
step_reader.SetMatMode(True)
step_reader.SetGDTMode(True)

status = step_reader.ReadFile(stp_path)
if status == IFSelect_RetDone:
    step_reader.Transfer(doc)
else:
	raise IOError("STEP file could not be read.")

In [3]:
#tdfdata=doc.GetData()
#tdfroot = tdfdata.Root()
# poly=BRep_Tool(tdfroot, )
shapes = XCAFDoc_DocumentTool().ShapeTool(doc.Main())
freelabels = TDF_LabelSequence()
shapes.GetFreeShapes(freelabels)
#rootShape = shapes.GetShape(tdfroot)
freeShape = shapes.GetShape(freelabels.Value(1))
BRepMesh_IncrementalMesh(freeShape, 0.01, True)
# export the document
vrml_exporter = VrmlAPI_Writer()
#vrml_exporter.Write(freeShape, 'tmp.wrl')
vrml_exporter.WriteDoc(doc, stp_filename + ".wrl", 1.0)

True

In [4]:
import subprocess as proc
proc.run(['chmod', 'a+x', '../bin/tovrmlx3d'])
#out = proc.run(['../bin/tovrmlx3d', '--encoding', 'xml', stp_filename + '.wrl'], stdout=open(stp_filename + '.x3d','wb'))
out = proc.run(['../bin/tovrmlx3d', '--encoding', 'xml', stp_filename + '.wrl'], stdout=proc.PIPE)

In [5]:
x3domHTML = '''<script type='text/javascript' src='https://www.x3dom.org/download/dev/x3dom-full.debug.js'> </script> 
<link rel='stylesheet' type='text/css' href='https://www.x3dom.org/download/dev/x3dom.css'></link>'''
x3domXML = ET.tostring(ET.XML(out.stdout), encoding="unicode", short_empty_elements=False)

In [6]:
HTML(x3domHTML+x3domXML)

In [ ]:
x3domXML